In [1]:
from tsfel.feature_extraction import features as tsfelf
import numpy as np
import os
from scipy import signal
import pandas as pd
import glob
import sklearn
from scipy.io import wavfile
from scipy.signal import decimate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
source_samp_rate=44100
decimation_factor = 4
#actual samples per second. When we record 1 second of audio we get only something like 0.9 seconds because of the chunk size
chunk=4096
single_audio_frame_size=10*chunk

In [3]:
def hpf(arr,fs,lf,order):
    wbut = lf
    bbut, abut = signal.butter(order,wbut, btype='highpass', fs=fs)
    return signal.filtfilt(bbut, abut, arr)

def get_features(sig,label,fs = 44100):
    
    s_f = hpf(sig, fs, 65, 2)

    Zero_Cross = tsfelf.zero_cross(sig)
    Zero_Cross_filt = tsfelf.zero_cross(s_f)
            
    Spectral_centroid = tsfelf.spectral_centroid(sig, fs)
    Spectral_centroid_filt = tsfelf.spectral_centroid(s_f, fs)

    Spectral_spread = tsfelf.spectral_spread(sig, fs)
    Spectral_spread_filt = tsfelf.spectral_spread(s_f, fs) 
            
    STD = np.std(sig)
    STD_filt = np.std(s_f)
    
    RMS = tsfelf.rms(sig)
    RMS_filt= tsfelf.rms(s_f)
    
    kurt_filt = tsfelf.kurtosis(s_f)
    
    return [label, RMS, RMS_filt, Zero_Cross, Zero_Cross_filt, Spectral_centroid, 
     Spectral_centroid_filt, Spectral_spread, Spectral_spread_filt, STD, STD_filt, kurt_filt]

In [4]:
def get_features_from_folder_3s_samples(folder_path,label,decimation_factor):
    files=glob.glob(folder_path+"/*.wav")
    #print(files)
    features_array = []
    for file_path in files:
        samplerate, sdata=wavfile.read(file_path)
        decimated_data=decimate(sdata,decimation_factor)
        sample1=decimated_data[0:int(single_audio_frame_size/decimation_factor*3)]
        print(len(sample1))
        features1=get_features(sample1,label,samplerate/decimation_factor)
        features_array.append(features1)
    return features_array

In [5]:
cwd = os.getcwd()
directory_contents = os.listdir(cwd)
list_of_directories = []

for item in directory_contents:
    if os.path.isdir(item):
        list_of_directories.append(item)

all_features=[]
for folder in list_of_directories:
    class_features = get_features_from_folder_3s_samples(cwd+"/"+folder,folder,decimation_factor)
    all_features=all_features+class_features

columnlist=["Label","RMS", "RMS_filt","Zero_Cross", "Zero_Cross_filt", "Spectral_centroid", "Spectral_centroid_filt", "Spectral_spread", \
                        "Spectral_spread_filt", "STD", "STD_filt", "kurt_filt"]
roadtype_features_df=pd.DataFrame(all_features,columns=columnlist)
filename="testset df-"+str(decimation_factor)+"_wl-3s_"+"09.03.2022.csv"
roadtype_features_df.to_csv(filename,index=False)
roadtype_features_df.head()





30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720
30720


,Label,RMS,RMS_filt,Zero_Cross,Zero_Cross_filt,Spectral_centroid,Spectral_centroid_filt,Spectral_spread,Spectral_spread_filt,STD,STD_filt,kurt_filt
0,Dry asphalt,1656.565722,1600.167307,8157,8525,1506.772529,1555.546968,924.469831,897.328947,1656.565683,1600.166731,1.043305
1,Dry asphalt,2100.432065,2026.851128,8541,8874,1533.298543,1582.089469,906.832116,869.428956,2100.431968,2026.851089,0.858933
2,Dry asphalt,3718.843713,3473.077181,7533,8086,1411.734137,1482.098892,906.843966,880.591185,3718.843703,3473.076528,0.637699
3,Dry asphalt,5185.855234,4757.537756,7418,7962,1390.176407,1459.457821,928.271038,891.621363,5185.855176,4757.537572,0.442316
4,Dry asphalt,5717.340091,5423.691878,7917,8358,1453.646119,1510.036542,899.999024,863.666004,5717.338862,5423.690805,0.473171
